# Deep learning notebook

This notebook contains the process for the deep learning part of the project. The idea is to achieve an n-dimensional representation of each text and get an average for a given time period in z-space, which can then be used for prediction.

In [1]:
from datetime import datetime

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.decomposition import PCA
from transformers import BertModel, BertTokenizer

c:\Users\samue\miniconda3\envs\capstone\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
tok = BertTokenizer.from_pretrained("KB/bert-base-swedish-cased")
model = BertModel.from_pretrained("KB/bert-base-swedish-cased")

Import data

In [2]:
df = pd.read_csv("../dataset/lawline_data.csv")

Remove newlines by splitting and combining texts. Then turn each document into a list of sentences.

In [ ]:
df["text_clean"] = [" ".join(str(item).split()) for item in df["text"]]

In [ ]:
doc_as_list = [item.split(". ") for item in df["text_clean"]]

In [ ]:
doc_as_list = [
    [sent if sent[-1] in ["!", "?"] else sent + "." for sent in doc]
    for doc in doc_as_list
]

Turn each document into a word embedding using BERT

In [ ]:
def get_embedding(document, tokenizer, model):
    results = tokenizer(
        document, max_length=512, truncation=True, padding=True, return_tensors="pt"
    )

    sentence_embs = model(**results)[1].detach().numpy()
    doc_emb = np.mean(sentence_embs, axis=0)

    return doc_emb

In [ ]:
emb_from_doc = [get_embedding(doc, tok, model) for doc in doc_as_list]

In [ ]:
emb_array = np.array(emb_from_doc)

In [ ]:
emb_df = pd.DataFrame(emb_array)

In [ ]:
emb_df.to_csv("../dataset/embeddings.csv", index=False)

Perform PCA to reduce dimensions of BERT embeddings.

In [ ]:
# Create the PCA instance
pca = PCA(n_components=0.95)

# Fit on data
pca.fit(emb_array)

# Access values and vectors
print(pca.explained_variance_)

# Transform data
pca_small = pca.transform(emb_array)

In [ ]:
df_reduced = pd.DataFrame(pca_small)

In [ ]:
df_reduced.shape

Add dates to DF to down-sample

In [ ]:
df_reduced["date"] = [
    datetime.strptime(date_str, "%d/%m/%Y") for date_str in df["date"]
]

In [ ]:
data_down = df_reduced.copy()
data_down = data_down.resample("M", on="date").mean()

In [ ]:
# include index to keep datetime
data_down.to_csv("../dataset/emb_down.csv")